In [ ]:
%matplotlib inline
from importlib import reload
import sys
sys.path.append("..")
from pathlib import Path
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from skimage import io
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import pythd

In [ ]:
DATA_DIR = Path("..") / Path("data")

POST_IMAGE_NAME = "hurricane-florence_00000419_post_disaster.png"
POST_IMAGE_PATH = DATA_DIR / POST_IMAGE_NAME
PRE_IMAGE_PATH = DATA_DIR / POST_IMAGE_NAME.replace("post", "pre")
POST_LABEL_PATH = DATA_DIR / POST_IMAGE_NAME.replace("png", "json")

In [ ]:
pre_img = io.imread(str(PRE_IMAGE_PATH))
post_img = io.imread(str(POST_IMAGE_PATH))

In [ ]:
reload(pythd)
reload(pythd.image)

pre_chipped = pythd.image.ChippedImage.FromRGB(pre_img, 16, 16)
pre_data = pre_chipped.get_data()
print(pre_data.shape)

In [ ]:
_ = pythd.image.plot_images((pre_img, post_img))

In [ ]:
filt = pythd.filter.ScikitLearnFilter(PCA, n_components=2)
f_x = filt(pre_data)
print(f_x.shape)

In [ ]:
reload(pythd)
reload(pythd.thd)
reload(pythd.complex)
reload(pythd.cover)
reload(pythd.mapper)
reload(pythd.clustering)

clustering = pythd.clustering.HierarchicalClustering()
cover = pythd.cover.IntervalCover.EvenlySpacedFromValues(f_x, 10, 0.3)
thd = pythd.thd.THD(pre_data, filt, cover, group_threshold=100)
thd.run()
root = thd.get_results()

In [ ]:
import igraph
g = root.as_igraph_graph()
vs = {
    "margin": 40,
    "bbox": (350, 350),
    "vertex_label": g.vs["name"],
    "vertex_label_size": 10,
    "vertex_size": 20,
    "vertex_label_dist": 1.5,
    "vertex_label_angle": 0,
    "layout": g.layout_reingold_tilford(root=[0])
}
igraph.plot(g, **vs)